## COAL INFRA MODEL

In [1]:
import Pkg; Pkg.add("Plots")

    Updating registry at `C:\Users\charl\.julia\registries\General.toml`
   Resolving package versions...
  No Changes to `C:\Users\charl\.julia\environments\v1.11\Project.toml`
  No Changes to `C:\Users\charl\.julia\environments\v1.11\Manifest.toml`


In [2]:
using Plots
using Pkg
using DataFrames, CSV
using CSV
using JuMP
using HiGHS
using Statistics

# JULIA Model 1

### Sets

- $Y$: Years  {0, 50}
- $T$: Time within a year {1...8760}
- $L$: Locations of coal power plants
- $G$: Generator technologies 
- $C$ : Converted capacity e.g., ($coal \to biomass$)


### Parameters

- $DEMAND_{t,y}$ : Demand in year $y$, time $t$  

- $INV_g$ : Investment cost for tech $g$ [\$/MW]  // $INV_{h \to g}^{conv}$ : Investment cost for repurpose tech g

- $FoM_g$ : Fixed O&M [\$/MW-yr]  

- $LIFE_g$ : Lifetime of tech $g$ [years]  

- $LEAD_g$ : Lead time before new build becomes available [years]  // $LEAD_{h \to g}^{conv}$ : Lead time before repurposed capacity becomes available

- $VC_g$ : Variable generation cost [\$/MWh]  

- $CF_{g,t}$ : Capacity factor of tech $g$ at time $t$  

- $Fp_g$ : Footprint coefficient of tech $g$ (e.g. land constraint)  

- $A_g$ : Area limit (location-specific, e.g. for renewables)  

- $PLANT_g$ : Existing capacity [MW]  

- $EF_g$ : Emission factor [tCO$_$/MWh]  

- $PCO2_y$ : Carbon price in year $y$ [\$/tCO$_$]   # change for each year increase

- $VOLL_{t,y}$ : Value of Lost Load [\$/MWh]  

- $r_g$ : Discount rate [WACC] of technology g


### Decision Variables

- $use_{t,y} \geq 0$ : Unserved energy demand at time $t, y$

- $gen_{g,t,y} \geq 0$ : Electricity generation from tech $g$  

- $build_{g,y} \geq 0$ : New build capacity for tech $g$  

- $ret_{g,y} \geq 0$ : Retired capacity for tech $g$  

- $cap_{g,y} \geq 0$ : Total available capacity of tech $g$

- $conv_{{h \to g},y} \geq 0$ : Total converted capacity of tech $g$ 


### Objective Function

$$
\min_{g,y,t} \;
\Bigg[
\sum_{g,y,t} \frac{1}{(1+r_g)^y} \, (VC_g + EF_g \cdot PCO2_y) \cdot gen_{g,t,y}
+ \sum_{g,y} \frac{1}{(1+r_g)^y} \, FoM_g \cdot cap_{g,y}
+ \sum_{g,y} \frac{1}{(1+r_g)^y} \, INV_g \cdot build_{g,y}
+ \sum_{h\to g \in C, y} \frac{1}{(1+r)^y} \, INV_{h \to g}^{conv} \cdot conv_{h \to g, y}
+ \sum_{y,t} \frac{1}{(1+r)^y} \, VOLL_{t,y} \cdot use_{t,y}
\Bigg]
$$


### Constraints

**Demand balance (with demand-side flexibility)**  
$$
\sum_g gen_{g,t,y} + use_{t,y} = DEMAND_{t,y}, \quad \forall t,y
$$

**Generation limited by capacity & capacity factor**  
$$
gen_{g,t,y} \leq cap_{g,y} \cdot CF_{g,t}, \quad \forall g,t,y
$$

**Initial capacity**  
$$
cap_{g,1} = PLANT_g, \quad \forall g
$$

**Capacity evolution (with lead time and retirement) and extension for conv**  
$$
cap_{g,y} =
\begin{cases}
PLANT_g, & y = 1 \\[6pt]
cap_{g,y-1} - ret_{g,y} - \sum_{k:g\to k} conv_{g\to k,y}, & 2 \leq y \leq LEAD_g \\[6pt]
cap_{g,y-1} + build_{g,y-LEAD_g} - ret_{g,y} + \sum_{h:h \to g,y} conv_{h\to g,y-{LEAD_g}^{conv}-1} - \sum_{k:g\to k} conv_{g\to k,y-1}, & y > LEAD_g
\end{cases}
$$

**Lifetime constraint**  # Remove don't need to retire in the horizon 
$$
ret_{g,y} \geq build_{g,y-LIFE_g}, \quad \forall g,y
$$

**Land/footprint constraint**  ## next step
$$
\sum_g Fp_g \cdot build_{g,y} \leq A_g, \quad \forall y
$$


**First Extension: Conversion Limit** 
$$
\sum_{h \to g \in c, y} conv_{h \to g, y} \leq cap_{h,y}
$$


### Julia JuMP implementation 


In [3]:
#sets
y0 = 0
Y  = y0:25 # years
T  = 1:8760                         
G  = [:coal, :gas, :solar, :wind, :biomass]
C = [(:coal, :biomass), (:coal, :gas)]

2-element Vector{Tuple{Symbol, Symbol}}:
 (:coal, :biomass)
 (:coal, :gas)

In [14]:
#parameters

r = Dict(
    :coal    => 0.04,   
    :biomass => 0.048,
    :solar   => 0.038,
    :wind    => 0.03,
    :gas     => 0.035
)

# Demand (MWh) per slice/year
Demand_Data = CSV.read("data/demand_25yrs_forecast.csv", DataFrame)
DEMAND = Dict((row.Year ,row.Hour) => row.Demand for row in eachrow(Demand_Data))

# Existing capacity (MW) in base year
PLANT = Dict(:coal=>1600.0, :gas=>800.0, :solar=>300.0, :wind=>300.0, :biomass=>1000.0) 

# Costs
INV = Dict(
    :coal    => 999_999_999.0,
    :gas     => 1_000_000.0, 
    :biomass => 5_391_000.0, 
    :solar   =>   750_000.0,
    :wind    => 1_000_000.0
) # $/MW

INV_C = Dict((:coal, :biomass)=>3_773_700.0, (:coal, :gas)=>700_000.0) # $/MW

FoM = Dict(
    :coal    => 120_000.0,
    :gas     => 28_000.0, 
    :biomass => 157_000.0,  #157000
    :solar   => 15_000.0,
    :wind    => 40_000.0
) # $/MW-yr

VC = Dict(
    :coal    => 24.2,
    :gas     => 27.6,   
    :biomass => 76.0,   #76  
    :solar   => 0.0,
    :wind    => 0.0
) # $/MWh
EF = Dict(
    :coal    => 0.774,
    :biomass => 0,   # 0.0 if carbon-neutral assumption or 1.502 if not neutral
    :gas     => 0.340,
    :solar   => 0.0,
    :wind    => 0.0
)# tCO₂/MWh

# Carbon price path
PCO2 = Dict(y => 90.44 for y in Y)  # $/tCO₂

# Value of lost load
VOLL = Dict((y,t) => 10_000.0 for y in Y, t in T) # $/MWh

# Capacity factors
CF_Data = CSV.read("data/CFData_Expanded.csv", DataFrame)

CF = Dict()
for row in eachrow(CF_Data)
    h = row.Hour
    CF[(:coal, h)]    = row.Coal
    CF[(:gas, h)]     = row.Gas
    CF[(:biomass, h)] = row.Biomass
    CF[(:wind, h)]    = row.Wind
    CF[(:solar, h)]   = row.Solar
end

# Lifetime and lead time (years)
LIFE = Dict(:coal=>30, :gas=>30, :solar=>30, :wind=>30, :biomass=>30) 
LEAD = Dict(:coal=>5,  :gas=>3,  :solar=>1,  :wind=>1,  :biomass=>2)
LEAD_C = Dict((:coal, :biomass)=>1, (:coal, :gas)=>2)


ErrorException: syntax: invalid keyword argument name ":solar" around c:\Users\charl\Documents\Julia\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X14sZmlsZQ==.jl:16

In [15]:
model = Model(HiGHS.Optimizer)

@variable(model, gen[g in G, t in T, y in Y] >= 0)    # generation [MWh]
@variable(model, cap[g in G, y in Y] >= 0)            # available capacity [MW]
@variable(model, build[g in G, y in Y] >= 0)          # new builds [MW]
@variable(model, retire[g in G, y in Y] >= 0)         # retirements [MW]
@variable(model, use[t in T, y in Y] >= 0)           # unserved load [MWh]
@variable(model, conv[c in C, y in Y] >= 0)          # conversions [MW] c = (h,g)

2-dimensional DenseAxisArray{VariableRef,2,...} with index sets:
    Dimension 1, [(:coal, :biomass), (:coal, :gas)]
    Dimension 2, 0:25
And data, a 2×26 Matrix{VariableRef}:
 conv[(:coal, :biomass),0]  …  conv[(:coal, :biomass),25]
 conv[(:coal, :gas),0]         conv[(:coal, :gas),25]

In [16]:
@objective(model, Min,
    # Variable generation cost + carbon
    sum((VC[g] + EF[g]*PCO2[y]) * gen[g,t,y] / (1+r[g])^(y) for g in G, t in T, y in Y) +
    # Fixed O&M
    sum(FoM[g] * cap[g,y] / (1+r[g])^(y) for g in G, y in Y) +
    # Investment
    sum(INV[g] * build[g,y] / (1+r[g])^(y) for g in G, y in Y) +
    # Investment (Conversion)
    sum(INV_C[c] * conv[c,y] / (1+0.05)^(y) for c in C, y in Y) + # Also used general WACC here need to check how WACC impacted by conversion 
    # Unserved demand (VOLL penalty)
    sum(VOLL[(y,t)] * use[t,y] / (1+0.05)^(y) for t in T, y in Y) # set r to 0.05 here as general WACC 
)

94.20056 gen[coal,1,0] + 90.57746153846153 gen[coal,1,1] + 87.09371301775147 gen[coal,1,2] + 83.74395482476103 gen[coal,1,3] + 80.52303348534713 gen[coal,1,4] + 77.42599373591071 gen[coal,1,5] + 74.44807089991413 gen[coal,1,6] + 71.58468355760975 gen[coal,1,7] + 68.83142649770168 gen[coal,1,8] + 66.18406394009776 gen[coal,1,9] + 63.63852301932477 gen[coal,1,10] + 61.190887518581505 gen[coal,1,11] + 58.837391844789906 gen[coal,1,12] + 56.57441523537491 gen[coal,1,13] + 54.39847618786049 gen[coal,1,14] + 52.30622710371201 gen[coal,1,15] + 50.294449138184625 gen[coal,1,16] + 48.36004724825444 gen[coal,1,17] + 46.50004543101388 gen[coal,1,18] + 44.711582145205654 gen[coal,1,19] + 42.99190590885159 gen[coal,1,20] + 41.33837106620345 gen[coal,1,21] + 39.74843371750332 gen[coal,1,22] + 38.21964780529165 gen[coal,1,23] + 36.74966135124197 gen[coal,1,24] + 35.33621283773267 gen[coal,1,25] + 94.20056 gen[coal,2,0] + 90.57746153846153 gen[coal,2,1] + 87.09371301775147 gen[coal,2,2] + 83.743954824

In [17]:
# Demand balance
@constraint(model, [y in Y, t in T],
    sum(gen[g,t,y] for g in G) + use[t,y] == DEMAND[(y,t)]
)

# Generation limited by capacity × CF
@constraint(model, [g in G, y in Y, t in T],
    gen[g,t,y] <= cap[g,y] * CF[(g,t)]
)

# Initial capacity
@constraint(model, [g in G],
    cap[g,y0] == PLANT[g]
)


# Lifetime constraint
for g in G, y in Y
    if y - y0 >= LIFE[g]
        @constraint(model, retire[g,y] >= build[g,y-LIFE[g]])
    end
end

# Capacity dynamics (with lead times) and conversion 

for g in G, (i,y) in enumerate(Y)
    if y > y0
        # base evolution: previous cap minus retire
        # add greenfield that finishes this year (if past lead)
        expr = @expression(model, cap[g,y-1] - retire[g,y] +
            ( (y - y0 >= LEAD[g]) ? build[g, y - LEAD[g]] : 0.0 )
        )

        # add conversions that ARRIVE this year (from any h -> g)
        for (h,gg) in C
            gg == g || continue
            if y - y0 >= LEAD_C[(h,gg)]
                expr +=  conv[(h,gg), y - LEAD_C[(h,gg)]]
            end
        end

        # subtract conversions that START this year going AWAY from g (g -> k)
        for (hh,k) in C
            hh == g || continue
            expr -= conv[(hh,k), y]
        end

        @constraint(model, cap[g,y] == expr)
    end
end

# Conversion constraint

for (h,_) in C, y in Y
    if y > y0
        @constraint(model, sum(conv[(h,g), y] for (hh,g) in C if hh == h) <= cap[h, y-1])
    else
        @constraint(model, sum(conv[(h,g), y] for (hh,g) in C if hh == h) == 0)
    end
end


In [ ]:
optimize!(model)

println("Objective value: ", objective_value(model))

In [ ]:
Gv = collect(G)  # ensure deterministic order (avoid Set iteration order)
Yv = collect(Y)

cap_df = DataFrame(
    tech = repeat(Gv, outer=length(Yv)),      # tech changes fastest within each year block
    year = repeat(Yv, inner=length(Gv)),      # year changes slowest
    MW   = [value(cap[g,y]) for y in Yv for g in Gv]
)

build_df = DataFrame(
    tech = repeat(Gv, outer=length(Yv)),
    year = repeat(Yv, inner=length(Gv)),
    MW   = [value(build[g,y]) for y in Yv for g in Gv]
)

retire_df = DataFrame(
    tech = repeat(Gv, outer=length(Yv)),
    year = repeat(Yv, inner=length(Gv)),
    MW   = [value(retire[g,y]) for y in Yv for g in Gv]
)


In [ ]:
using DataFrames, CSV, JuMP

# Helper: safe value() that returns 0.0 for missing/tiny/NaN
_v(x) = try
    v = value(x)
    (isfinite(v) && abs(v) > 1e-9) ? v : 0.0
catch
    0.0
end

Gv = collect(G); Yv = collect(Y); Tv = collect(T)

gen_df = DataFrame(tech=String[], year=Int[], hour=Int[], gen_MWh=Float64[])
for y in Yv, t in Tv, g in Gv
    push!(gen_df, (string(g), y, t, _v(gen[g,t,y])))
end
CSV.write("results_generation_hourly.csv", gen_df)

gen_annual_df = combine(groupby(gen_df, [:tech,:year]), :gen_MWh => sum => :gen_MWh)
CSV.write("results_generation_annual.csv", gen_annual_df)

cap_df   = DataFrame(tech=String[], year=Int[], cap_MW=Float64[])
build_df = DataFrame(tech=String[], year=Int[], build_MW=Float64[])
ret_df   = DataFrame(tech=String[], year=Int[], retire_MW=Float64[])
for y in Yv, g in Gv
    push!(cap_df,   (string(g), y, _v(cap[g,y])))
    push!(build_df, (string(g), y, _v(build[g,y])))
    push!(ret_df,   (string(g), y, _v(retire[g,y])))
end
CSV.write("results_capacity.csv", cap_df)
CSV.write("results_build.csv",    build_df)
CSV.write("results_retire.csv",   ret_df)

shed_df = DataFrame(year=Int[], hour=Int[], unserved_MWh=Float64[])
for y in Yv, t in Tv
    push!(shed_df, (y, t, _v(use[t,y])))
end
CSV.write("results_unserved_hourly.csv", shed_df)

has_conv = @isdefined(C) && @isdefined(conv) && !isempty(C)

conv_flows_df = DataFrame(h=String[], g=String[], year=Int[], conv_start_MW=Float64[])
if has_conv
    for (h,g) in C, y in Yv
        push!(conv_flows_df, (string(h), string(g), y, _v(conv[(h,g),y])))
    end
end
CSV.write("results_conversion_flows.csv", conv_flows_df)

rep_arrivals_df = DataFrame(g=String[], year=Int[], repurpose_arrive_MW=Float64[])
if has_conv
    for y in Yv, (h,g) in C
        lead = get(LEAD_C, (h,g), 0)
        arrivals = (y - y0) >= lead ? _v(conv[(h,g), y - lead]) : 0.0
        push!(rep_arrivals_df, (string(g), y, arrivals))
    end
end
CSV.write("results_repurposed_arrivals.csv", rep_arrivals_df)

println("CSVs written (single-region):")
println(" - results_generation_hourly.csv")
println(" - results_generation_annual.csv")
println(" - results_capacity.csv")
println(" - results_build.csv")
println(" - results_retire.csv")
println(" - results_unserved_hourly.csv")
println(" - results_conversion_flows.csv")
println(" - results_repurposed_arrivals.csv")


# Model 2 Julia Location Integration



Model the same just building in location indexing and sets 

### Sets

- $Y$: Years  {0, 25}
- $T$: Time within a year {1...8760}
- $L$: Locations of coal power plants
- $G$: Generator technologies 
- $C$ : Converted capacity e.g., ($coal \to biomass$)


### Parameters

- $DEMAND_{t,y}$ : Demand in year $y$, time $t$  

- $INV_g$ : Investment cost for tech $g$ [\$/MW]  // $INV_{h \to g}^{conv}$ : Investment cost for repurpose tech g

- $FoM_g$ : Fixed O&M [\$/MW-yr]  

- $LIFE_g$ : Lifetime of tech $g$ [years]  

- $LEAD_g$ : Lead time before new build becomes available [years]  // $LEAD_{h \to g}^{conv}$ : Lead time before repurposed capacity becomes available

- $VC_g$ : Variable generation cost [\$/MWh]  

- $CF_{g,t,l}$ : Capacity factor of tech $g$ at time $t$ at location $l$

- $Fp_g$ : Footprint coefficient of tech $g$ (e.g. land constraint)  

- $A_l$ : Area limit (location-specific, e.g. for renewables)  

- $PLANT_{g_l}$ : Existing capacity [MW]  

- $EF_g$ : Emission factor [tCO$_$/MWh]  

- $PCO2_y$ : Carbon price in year $y$ [\$/tCO$_$]  

- $VOLL_{t,y}$ : Value of Lost Load [\$/MWh]  

- $r_g$ : Discount rate [WACC] of technology g

### Decision Variables

- $use_{t,y} \geq 0$ : Unserved energy demand at time $t, y$

- $gen_{g,t,y,l} \geq 0$ : Electricity generation from tech $g$ at location $l$

- $build_{g,y,l} \geq 0$ : New build capacity for tech $g$ at location $l$

- $ret_{g,y,l} \geq 0$ : Retired capacity for tech $g$  

- $cap_{g,y,l} \geq 0$ : Total available capacity of tech $g$

- $conv_{{h \to g},y, l} \geq 0$ : Total converted capacity of tech $g$ at location l


### Objective Function

$$
\min_{g,l,y,t} \;
\Bigg[
\sum_{g,l,y,t} \frac{1}{(1+r_g)^y} \, (VC_g + EF_g \cdot PCO2_y) \cdot gen_{g,l,t,y}
+ \sum_{g,l,y} \frac{1}{(1+r_g)^y} \, FoM_g \cdot cap_{g,l,y}
+ \sum_{g,l,y} \frac{1}{(1+r_g)^y} \, INV_g \cdot build_{g,l,y}
+ \sum_{h\to g \in C,l,y} \frac{1}{(1+r)^y} \, INV_{h \to g}^{conv} \cdot conv_{h \to g,l, y}
+ \sum_{y,t} \frac{1}{(1+r)^y} \, VOLL_{t,y} \cdot use_{t,y}
\Bigg]
$$


### Constraints

**Demand balance (with demand-side flexibility)**  
$$
\sum_{g,l} gen_{g,l,t,y} + use_{t,y} = DEMAND_{t,y}, \quad \forall t,y
$$

**Generation limited by capacity & capacity factor**  
$$
gen_{g,t,y,l} \leq cap_{g,y,l} \cdot CF_{g,t,l}, \quad \forall g,t,y,l
$$

**Initial capacity**  
$$
cap_{g,1,l} = PLANT_{g,l}, \quad \forall g,l
$$

**Capacity evolution (with lead time and retirement) and extension for conv**  
$$
cap_{g,l,y} =
\begin{cases}
PLANT_g,l, & y = 1 \\[6pt]
cap_{g,l,y-1} - ret_{g,l,y} - \sum_{k:g\to k,} conv_{g\to k,l,y}, & 2 \leq y \leq LEAD_g \\[6pt]
cap_{g,l,y-1} + build_{g,l,y-LEAD_g} - ret_{g,l,y} + \sum_{h:h \to g,y} conv_{h\to g,y-{LEAD_g}^{conv}} - \sum_{k:g\to k} conv_{g\to k,l,y}, & y > LEAD_g
\end{cases}
$$

**Land/footprint constraint** 
$$
\sum_g Fp_g \cdot build_{g,y,l} \leq A_l, \quad \forall y
$$


**Conversion Limit** 
$$
\sum_{h \to g \in c, y} conv_{h \to g, y} \leq cap_{g,y}
$$


In [9]:
#sets
y0 = 0
Y  = y0:25 # years
T  = 1:8760                         
G  = [:coal, :gas, :solar, :wind, :biomass]
C = [(:coal, :biomass), (:coal, :gas)]
L = [:z1,:z2,:z3]

3-element Vector{Symbol}:
 :z1
 :z2
 :z3

In [10]:
#parameters

r = Dict(
    :coal    => 0.04,   
    :biomass => 0.048,
    :solar   => 0.038,
    :wind    => 0.03,
    :gas     => 0.035
)

# Demand (MWh) per slice/year
Demand_Data = CSV.read("data/demand_25yrs_forecast.csv", DataFrame)
DEMAND = Dict((row.Year ,row.Hour) => row.Demand for row in eachrow(Demand_Data))

# Existing capacity (MW) in base year

PLANT_g = Dict(:coal=>4000.0, :gas=>2000.0, :solar=>800.0, :wind=>1200.0, :biomass=>1000.0)
share = Dict(:z1=>0.4, :z2=>0.35, :z3=>0.25)
PLANT = Dict{Tuple{Symbol,Symbol},Float64}()
for (g, capg) in PLANT_g, l in L
    PLANT[(g,l)] = capg * share[l]
end

# Costs
INV = Dict(
    :coal    => 999_999_999.0,
    :gas     => 1_000_000.0, 
    :biomass => 5_391_000.0, 
    :solar   =>   750_000.0,
    :wind    => 1_000_000.0
) # $/MW

INV_C = Dict((:coal, :biomass)=>3_773_700.0, (:coal, :gas)=>700_000.0) # $/MW

FoM = Dict(
    :coal    => 120_000.0,
    :gas     => 28_000.0, 
    :biomass => 157_000.0,  
    :solar   => 15_000.0,
    :wind    => 40_000.0
) # $/MW-yr

VC = Dict(
    :coal    => 24.2,
    :gas     => 27.6,   
    :biomass => 76.0,     
    :solar   => 0.0,
    :wind    => 0.0
) # $/MWh
EF = Dict(
    :coal    => 0.774,
    :biomass => 1.502,   # or 0.0 if carbon-neutral assumption
    :gas     => 0.340,
    :solar   => 0.0,
    :wind    => 0.0
)# tCO₂/MWh

# Carbon price path
PCO2 = Dict(y => 90.44 for y in Y)  # $/tCO₂

# Value of lost load
VOLL = Dict((y,t) => 10_000.0 for y in Y, t in T) # $/MWh

# Capacity factors
CF_Data = CSV.read("data/CFData_Expanded.csv", DataFrame)

CF_b = Dict()
for row in eachrow(CF_Data)
    h = row.Hour
    CF_b[(:coal, h)]    = row.Coal
    CF_b[(:gas, h)]     = row.Gas
    CF_b[(:biomass, h)] = row.Biomass
    CF_b[(:wind, h)]    = row.Wind
    CF_b[(:solar, h)]   = row.Solar
end

mult = Dict(:z1=>1.00, :z2=>0.90, :z3=>1.10)   # e.g., z3 is windier/sunnier than z2
CF = Dict{Tuple{Symbol,Symbol,Int},Float64}()
for (g,t) in keys(CF_b), l in L
    base = CF_b[(g,t)]
    m = (g in (:wind,:solar)) ? mult[l] : 1.0
    CF[(g,l,t)] = clamp(base * m, 0.0, 1.0)
end

# Lifetime and lead time (years)
LIFE = Dict(:coal=>30, :gas=>30, :solar=>30, :wind=>30, :biomass=>30) 
LEAD = Dict(:coal=>5,  :gas=>3,  :solar=>1,  :wind=>1,  :biomass=>2)
LEAD_C = Dict((:coal, :biomass)=>1, (:coal, :gas)=>2)

FP = Dict(:solar=>0.008, :wind=>0.020, :biomass=>0.002, :coal=>0.0, :gas=>0.0)
AREA = Dict(:z1=>120.0, :z2=>80.0, :z3=>55.0)

Dict{Symbol, Float64} with 3 entries:
  :z3 => 55.0
  :z1 => 120.0
  :z2 => 80.0

In [11]:
model_LOC = Model(HiGHS.Optimizer)

@variable(model_LOC, gen[g in G, l in L, t in T, y in Y] >= 0)    # generation [MWh]
@variable(model_LOC, cap[g in G, l in L, y in Y] >= 0)            # available capacity [MW]
@variable(model_LOC, build[g in G, l in L, y in Y] >= 0)          # new builds [MW]
@variable(model_LOC, retire[g in G, l in L, y in Y] >= 0)         # retirements [MW]
@variable(model_LOC, use[t in T, y in Y] >= 0)           # unserved load [MWh]
@variable(model_LOC, conv[c in C, l in L, y in Y] >= 0)          # conversions [MW] c = (h,g)

3-dimensional DenseAxisArray{VariableRef,3,...} with index sets:
    Dimension 1, [(:coal, :biomass), (:coal, :gas)]
    Dimension 2, [:z1, :z2, :z3]
    Dimension 3, 0:25
And data, a 2×3×26 Array{VariableRef, 3}:
[:, :, 0] =
 conv[(:coal, :biomass),z1,0]  …  conv[(:coal, :biomass),z3,0]
 conv[(:coal, :gas),z1,0]         conv[(:coal, :gas),z3,0]

[:, :, 1] =
 conv[(:coal, :biomass),z1,1]  …  conv[(:coal, :biomass),z3,1]
 conv[(:coal, :gas),z1,1]         conv[(:coal, :gas),z3,1]

[:, :, 2] =
 conv[(:coal, :biomass),z1,2]  …  conv[(:coal, :biomass),z3,2]
 conv[(:coal, :gas),z1,2]         conv[(:coal, :gas),z3,2]

...

[:, :, 23] =
 conv[(:coal, :biomass),z1,23]  …  conv[(:coal, :biomass),z3,23]
 conv[(:coal, :gas),z1,23]         conv[(:coal, :gas),z3,23]

[:, :, 24] =
 conv[(:coal, :biomass),z1,24]  …  conv[(:coal, :biomass),z3,24]
 conv[(:coal, :gas),z1,24]         conv[(:coal, :gas),z3,24]

[:, :, 25] =
 conv[(:coal, :biomass),z1,25]  …  conv[(:coal, :biomass),z3,25]
 conv[(:coal, :gas

In [12]:
@objective(model_LOC, Min,
    # Variable gen cost + carbon
    sum((VC[g] + EF[g]*PCO2[y]) * gen[g,l,t,y] / (1 + r[g])^(y - y0)
        for g in G, l in L, t in T, y in Y) +

    # Fixed O&M
    sum(FoM[g] * cap[g,l,y] / (1 + r[g])^(y - y0)
        for g in G, l in L, y in Y) +

    # Investment (greenfield)
    sum(INV[g] * build[g,l,y] / (1 + r[g])^(y - y0)
        for g in G, l in L, y in Y) +

    # Investment (conversion) used global WACC r = 0.05
    sum(INV_C[c] * conv[c,l,y] / (1 + 0.05)^(y - y0)
        for c in C, l in L, y in Y) +

    # Unserved demand (VOLL penalty) — system-wide discount
    sum(VOLL[(y,t)] * use[t,y] / (1 + 0.05)^(y - y0)
        for t in T, y in Y)
)


94.20056 gen[coal,z1,1,0] + 90.57746153846153 gen[coal,z1,1,1] + 87.09371301775147 gen[coal,z1,1,2] + 83.74395482476103 gen[coal,z1,1,3] + 80.52303348534713 gen[coal,z1,1,4] + 77.42599373591071 gen[coal,z1,1,5] + 74.44807089991413 gen[coal,z1,1,6] + 71.58468355760975 gen[coal,z1,1,7] + 68.83142649770168 gen[coal,z1,1,8] + 66.18406394009776 gen[coal,z1,1,9] + 63.63852301932477 gen[coal,z1,1,10] + 61.190887518581505 gen[coal,z1,1,11] + 58.837391844789906 gen[coal,z1,1,12] + 56.57441523537491 gen[coal,z1,1,13] + 54.39847618786049 gen[coal,z1,1,14] + 52.30622710371201 gen[coal,z1,1,15] + 50.294449138184625 gen[coal,z1,1,16] + 48.36004724825444 gen[coal,z1,1,17] + 46.50004543101388 gen[coal,z1,1,18] + 44.711582145205654 gen[coal,z1,1,19] + 42.99190590885159 gen[coal,z1,1,20] + 41.33837106620345 gen[coal,z1,1,21] + 39.74843371750332 gen[coal,z1,1,22] + 38.21964780529165 gen[coal,z1,1,23] + 36.74966135124197 gen[coal,z1,1,24] + 35.33621283773267 gen[coal,z1,1,25] + 94.20056 gen[coal,z1,2,0] +

In [13]:
# Demand balance
@constraint(model_LOC, [y in Y, t in T],
    sum(gen[g,l,t,y] for g in G, l in L) + use[t,y] == DEMAND[(y,t)]
)

# Generation limited by capacity × CF
@constraint(model_LOC, [g in G,l in L, y in Y, t in T],
    gen[g,l,t,y] <= cap[g,l,y] * CF[(g,l,t)]
)

# Initial capacity
@constraint(model_LOC, [g in G, l in L],
    cap[g,l,y0] == PLANT[(g,l)]
)


# Lifetime constraint
for g in G, l in L, y in Y
    if y - y0 >= LIFE[g]
        @constraint(model_LOC, retire[g,l,y] >= build[g,l,y-LIFE[g]])
    end
end

# Capacity dynamics (with lead times) and conversion 

for g in G, l in L, (i,y) in enumerate(Y)
    if y > y0
        # carry forward stock and retire this year
        expr = @expression(model_LOC, cap[g,l,y-1] - retire[g,l,y] +
            ((y - y0 >= LEAD[g]) ? build[g,l, y - LEAD[g]] : 0.0)
        )

        # conversions ARRIVING to (g,l) this year: add after conversion lead
        for (h,gg) in C
            gg == g || continue
            if y - y0 >= LEAD_C[(h,gg)]
                expr += conv[(h,gg), l, y - LEAD_C[(h,gg)]]
            end
        end

        # conversions STARTING this year that LEAVE (g,l): subtract now (construction gap)
        for (hh,k) in C
            hh == g || continue
            expr -= conv[(hh,k), l, y]
        end

        @constraint(model_LOC, cap[g,l,y] == expr)
    end
end


# Conversion constraint

for (h,_) in C, l in L, y in Y
    if y > y0
        @constraint(model_LOC,
            sum(conv[(h,g), l, y] for (hh,g) in C if hh == h) <= cap[h,l, y-1]
        )
    else
        @constraint(model_LOC,
            sum(conv[(h,g), l, y] for (hh,g) in C if hh == h) == 0
        )
    end
end

# Area constraint

@constraint(model_LOC, [l in L, y in Y],
    sum(FP[g] * cap[g,l,y] for g in G) <= AREA[l]
)


2-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape},2,...} with index sets:
    Dimension 1, [:z1, :z2, :z3]
    Dimension 2, 0:25
And data, a 3×26 Matrix{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape}}:
 0.008 cap[solar,z1,0] + 0.02 cap[wind,z1,0] + 0.002 cap[biomass,z1,0] <= 120  …  0.008 cap[solar,z1,25] + 0.02 cap[wind,z1,25] + 0.002 cap[biomass,z1,25] <= 120
 0.008 cap[solar,z2,0] + 0.02 cap[wind,z2,0] + 0.002 cap[biomass,z2,0] <= 80      0.008 cap[solar,z2,25] + 0.02 cap[wind,z2,25] + 0.002 cap[biomass,z2,25] <= 80
 0.008 cap[solar,z3,0] + 0.02 cap[wind,z3,0] + 0.002 cap[biomass,z3,0] <= 55      0.008 cap[solar,z3,25] + 0.02 cap[wind,z3,25] + 0.002 cap[biomass,z3,25] <= 55

In [ ]:
optimize!(model_LOC)

println("Objective value: ", objective_value(model_LOC))